# "Conhecendo a Estrutura do Plotly (Parte 3)"
> "O retorno do rei!"

- toc: true
- branch: master
- badges: false
- comments: true
- author: Augusto dos Santos Pereira
- categories: [Plotly, Gráficos, Python]

In [1]:
# hide_input
# This cell is required for the export to HTML to work.
import plotly.io as pio
#pio.renderers.default = 'plotly_mimetype+notebook'
pio.renderers.default = 'colab'
# Default is plotly_mimetype+notebook, but jekyll fails to parse plotly_mimetype.
#pio.renderers.default = 'notebook_connected'
# Uncomment below to avoid using a CDN for plotly.js
# pio.renderers.default = 'notebook'

# Inject the missing require.js dependency.
from IPython.display import display, HTML
#js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
#display(HTML(js))

## A Trilogia

Esta postagem é o fim de uma trilogia em que procurei apresentar a **estrutura do pacote [Plotly](https://plotly.com/python/)**, a ferramenta do ecossistema Python para dados que mais tenho utilizado na elaboração de produtos analíticos - gráficos, cartogramas e diagramas. 

Até aqui, vimos cinco dos sete subpacotes principais do Plotly. Na **[primeira parte](https://augustogeog.github.io/augustogeog/plotly/gr%C3%A1ficos/python/2021/05/31/introdu%C3%A7%C3%A3o-plotly.html)** da série
vimos que **Plotly Data** apresenta funções para carregamento de datasets-exemplos como Pandas DataFrame, que **Plotly Express** compõe uma interface de alto nível de abstração que permite fazer gráficos arrojados com uma única função, enquanto **Plotly Colors** agrega funções que geram diferentes tipos de paletas de cores cíclicas, divergentes, sequenciais e qualitativas.

Na **[segunda parte](https://augustogeog.github.io/augustogeog/plotly/gr%C3%A1ficos/python/2021/06/05/estrutura-plotly-part2.html)**, vimos que  **Graph Objects**, por sua vez, são objetos que envolvem baixo nível de abstração, permitindo maior nível de personalização de gráficos, ao passo que o pacote **Subplots** é voltado para a facilitação da geração de figuras compostas por diversos gráficos.

Agora, mente aberta, espinha erata e o coração tranquilo, pois vamos ver um pouco do que é possível fazer com Plotly **Plotly IO** e **Figure Factory**.

> Important: A estrutura do pacote e os exemplos de código utilizados nesta série se referem à versão 4.14.3 do Plotly.

## Plotly IO

IO é uma sigla bastante recorrente em tecnologias computacionais. Refere-se a input e output. Nas bibliotecas do ecossistema Python para tecnologias de dados, IO usualmente se refere a importação e exportação de dados. Assim, no subpacote **Plotly IO**, temos diversas funções para importar e exportar dados estruturados para formação de Plotly Figures, com suas instruções para renderização de gráficos, cartogramas e diagramas.

Plotly IO apresenta funções para que possamos exportar os nossos gráficos para formatos de imagem (pio.write_image), para html (pio.write_html) e json (pio.write_json). O subpacote ainda nos permite escolher qual o tema padrão para _template_, ou seja, a aparência geral dos gráficos (`pio.templates.default = 'ggplot2'`, por exemplo, faz com que os gráficos tenha aparência do pacote mais utilizado em linguagem R). No namespace IO podemos também escolher o renderizador a ser utilizado (`pio.renderers.default = 'plotly_mimetype+notebook'`, para Jupyter Notebooks, e `pio.renderers.default = 'colab'` para Google Colab, por exemplo). 

Vejamos adiante como exportar os gráficos em formato JSON e HTML, o que pode ser útil para lançamento de serviços _web_

In [2]:
import plotly.io as pio # importando a biblioteca Plotly IO
import pandas as pd # importando pacote pandas para geração de DataFrame para dados tabulares
import plotly.express as px # importando subpacote Plotly Express
from plotly.data import wind

df = wind()

df

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
...,...,...,...
123,WSW,6+,0.1
124,W,6+,0.9
125,WNW,6+,2.2
126,NW,6+,1.5
